In [29]:
import keras
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.models import Model
import numpy as np
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import tensorflow as tf

#keras: v2.1.6 --- tf: v1.8.0
print('Keras version: ', keras.__version__)
print('Tensorflow version: ', tf.__version__)
%matplotlib inline

Keras version:  2.1.6
Tensorflow version:  1.8.0


In [30]:
def facial_recognition_model(dim):
    '''
    Facial recognition section of the all in one architecture based on the triplet
    probability embeddings paper architecture.  
    Takes in a 227, 227, 3 aligned facial images (alignment is based on the Hyperface paper)
    
    returns a facial recognition model which outputs a face desciptor of dimension 10548
    '''
    
    face_input_shape = (dim, dim, 3)
    facial_image_input = Input(face_input_shape)
    
    
    ### 
    # The top three convolutional layers (conv1-conv3) 
    # are initialized with the weights from the AlexNet model
    # trained on the ImageNet challenge dataset.
    ###
    
    #11x11 conv, 96 filters, stride=4
    facial_recognition = Conv2D(filters=96, kernel_size=(11,11), strides=(4,4),name = 'face_recognition_conv1')(facial_image_input)
    facial_recognition = BatchNormalization(axis=3, name = 'face_recognition_bn1')(facial_recognition)
    facial_recognition = Activation(PReLU())(facial_recognition)
    facial_recognition = MaxPooling2D(pool_size=(3,3), strides=(2,2), name = 'face_recognition_mp1')(facial_recognition)

    #5x5 conv, 256 filters, stride=2
    facial_recognition = Conv2D(filters=256, kernel_size=(5,5), strides=(2,2), name = 'face_recognition_conv2')(facial_recognition)
    facial_recognition = BatchNormalization(axis=3, name = 'face_recognition_bn2')(facial_recognition)
    facial_recognition = Activation(PReLU())(facial_recognition)
    facial_recognition = MaxPooling2D(pool_size=(3,3), strides=(2,2), name = 'face_recognition_mp2')(facial_recognition)
    
    #3x3 conv, 384 filters
    facial_recognition = Conv2D(filters=384, kernel_size=(3,3), padding='same',name = 'face_recognition_conv3')(facial_recognition)
    facial_recognition = Activation(PReLU())(facial_recognition)
    
    ## The layers conv4-conv7 do not downsample 
    #  the input thereby learning more complex
    ## higher dimensional representations.
    
    ##3x3 conv, 384 filters
    facial_recognition = Conv2D(filters=384, kernel_size=(3,3), padding='same',name = 'face_recognition_conv4')(facial_recognition)
    facial_recognition = Activation(PReLU())(facial_recognition)

    #3x3 conv, 512 filters
    facial_recognition = Conv2D(filters=512, kernel_size=(3,3), padding='same',name = 'face_recognition_conv5')(facial_recognition)
    facial_recognition = Activation(PReLU())(facial_recognition)

    #3x3 conv, 512 filters
    facial_recognition = Conv2D(filters=512, kernel_size=(3,3), padding='same', name = 'face_recognition_conv6')(facial_recognition)
    facial_recognition = Activation(PReLU())(facial_recognition)

    #3x3 conv, 512 filters
    facial_recognition = Conv2D(filters=512, kernel_size=(3,3), padding='same',name = 'face_recognition_conv7')(facial_recognition)
    facial_recognition = MaxPooling2D(pool_size=(3,3), strides=(2,2), name = 'face_recognition_mp7')(facial_recognition)
    facial_recognition = Activation(PReLU())(facial_recognition)
    
    ###
    # Since the network is used as a feature extractor, the last
    # layer fc8 is removed during deployment
    ###
    
    #fully-connected layer - 1024 units
    facial_recognition = Flatten()(facial_recognition)
    facial_recognition = Dropout(.5)(facial_recognition)
    facial_recognition = Dense(units=1024, activation=PReLU(), name = 'face_recognition_fc1')(facial_recognition)

    #fully-connected layer - 512 units 
    facial_recognition = Dense(units=512, activation=PReLU(), name = 'face_recognition_fc2')(facial_recognition)

    #fully-connected layer - 10548 units
    facial_recognition = Dense(units=10548, activation='softmax', name = 'face_recognition_fc3')(facial_recognition)

    facial_recog_model = Model(inputs= facial_image_input, outputs = facial_recognition, name = 'All_in_one_model')
    
    return facial_recog_model

In [31]:
face_recog = facial_recognition_model(227)

/home/nardos/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [32]:
face_recog.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
face_recognition_conv1 (Conv (None, 55, 55, 96)        34944     
_________________________________________________________________
face_recognition_bn1 (BatchN (None, 55, 55, 96)        384       
_________________________________________________________________
activation_50 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
face_recognition_mp1 (MaxPoo (None, 27, 27, 96)        0         
_________________________________________________________________
face_recognition_conv2 (Conv (None, 12, 12, 256)       614656    
_________________________________________________________________
face_recognition_bn2 (BatchN (None, 12, 12, 256)       1024      
__________